# General success rate

In [ ]:
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re

In [ ]:
ctx = snowflake.connector.connect(
      user = '',
     password = '',
     account = ''
 )

cursor = ctx.cursor()

In [ ]:
import pandas as pd
sql_success_rate_MLS = """"""
df_success_rate_MLS = pd.read_sql(sql_success_rate_MLS, con=ctx)

In [ ]:
#!pip install placekey
from placekey.api import PlacekeyAPI
from ast import literal_eval
placekey_api_key = "" # fill this in with your personal API key (do not share publicly)
pk_api = PlacekeyAPI(placekey_api_key)

In [ ]:
# adding required placekey api field
def get_standard_df(df,column_map: dict):
    # rename column name --> need to be consistant with standatdized api field name
    # see this link: https://docs.placekey.io/#350ed3a9-68db-4c47-9e20-19b430cb9ef1
    df_standard = df.rename(columns=column_map)
    
    # adding other fields
    df_standard['query_id'] = range(1, 1+len(df))
    df_standard['iso_country_code'] = "US"
    return df_standard

def get_placekey_df(stad_df, place_keys:tuple):
    # stad_df is the df containing standatdized api fields
    import json
    # convert query_id to str
    stad_df = stad_df.astype({"query_id": str})
    
    # dump df to json-format with selected key-value pairs
    data_jsoned = json.loads(stad_df.to_json(orient="records"))
    dict_filter = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
    result_dict=[dict_filter(i, place_keys) for i in data_jsoned]
    
    #request placekey api
    responses = pk_api.lookup_placekeys(result_dict, verbose=True)
    df_placekeys = pd.read_json(json.dumps(responses), dtype={'query_id':str})
    

    return df_placekeys

In [ ]:
column_map = {"CC_PROPERTY_ADDR_DISPLAY_1" : "street_address", 
              "CC_PROPERTY_ADDR_POSTAL_CODE": "postal_code",
              "CC_PROPERTY_ADDR_STATE": "region",
              "CC_PROPERTY_ADDR_CITY": "city"}
stad_df = get_standard_df(df_success_rate_MLS,column_map)
stad_df.reset_index(inplace=True)

place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
df_placekey = get_placekey_df(stad_df,place_keys)

df_success_rate_placekey_MLS = pd.concat([df_placekey,df_success_rate_MLS],axis=1)

In [ ]:
df_success_rate_placekey_MLS = pd.concat([df_placekey,df_success_rate_MLS],axis=1)

- Error rate in MLS table

In [ ]:
df_success_rate_placekey_MLS['error'].unique()

In [ ]:
len(df_success_rate_placekey_MLS[df_success_rate_placekey_MLS['error'].notnull()])/100000

In [ ]:
sql_success_rate_ASSESSOR = """"""
df_success_rate_ASSESSOR = pd.read_sql(sql_success_rate_ASSESSOR, con=ctx)

In [ ]:
df_success_rate_ASSESSOR

In [ ]:
column_map = {"SITUSFULLSTREETADDRESS" : "street_address", 
              "SITUSZIP5": "postal_code",
              "SITUSSTATE": "region",
              "SITUSCITY": "city"}
stad_df = get_standard_df(df_success_rate_ASSESSOR,column_map)
stad_df.reset_index(inplace=True)

place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
df_placekey = get_placekey_df(stad_df,place_keys)

df_success_rate_placekey_ASSESSOR = pd.concat([df_placekey,df_success_rate_ASSESSOR],axis=1)

In [ ]:
df_success_rate_placekey_ASSESSOR['error'].unique()

In [ ]:
len(df_success_rate_placekey_ASSESSOR[df_success_rate_placekey_ASSESSOR['error'].notnull()])/100000

In [ ]:
df_success_rate_placekey_ASSESSOR